In [1]:
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [2]:
class VGG(nn.Module):
    
    def __init__(self, num_classes=1000):
        super(VGG, self).__init__()
        self.conv1_1 = self.make_conv_relu(3, 64)
        
        self.conv2_1 = self.make_conv_relu(64, 128)

        self.conv3_1 = self.make_conv_relu(128, 256)
        self.conv3_2 = self.make_conv_relu(256, 256)

        self.conv4_1 = self.make_conv_relu(256, 512)
        self.conv4_2 = self.make_conv_relu(512, 512)

        self.conv5_1 = self.make_conv_relu(512, 512)
        self.conv5_2 = self.make_conv_relu(512, 512)
        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.relu = nn.ReLU()
        
        self.classifier = nn.Linear(512, num_classes)

    def make_conv_relu(self, in_channels, out_channel):
        layers = []
        layers += [nn.Conv2d(in_channels, out_channel, kernel_size=3, padding=1),  #using kernel size 3, padding 1 -> keep the spatial dimension of tensor
                   nn.ReLU()]
        return nn.Sequential(*layers)
        
    def forward(self, x):
        # x shape: [Batchsize, 3, 32, 32]
        
        x = self.conv1_1(x) # [Batchsize, 64, 32, 32]
        x = self.maxpool(x) # [Batchsize, 64, 16, 16]
        
        x = self.conv2_1(x) # [Batchsize, 128, 16, 16]
        x = self.maxpool(x) # [Batchsize, 128, 8, 8]
        
        x = self.conv3_1(x) # [Batchsize, 256, 8, 8]
        x = self.conv3_2(x) # [Batchsize, 256, 8, 8]
        x = self.maxpool(x) # [Batchsize, 256, 4, 4]
        
        x = self.conv4_1(x) # [Batchsize, 512, 4, 4]
        x = self.conv4_2(x) # [Batchsize, 512, 4, 4]
        x = self.maxpool(x) # [Batchsize, 512, 2, 2]
        
        x = self.conv5_1(x) # [Batchsize, 512, 2, 2]
        x = self.conv5_2(x) # [Batchsize, 512, 2, 2]
        x = self.maxpool(x) # [Batchsize, 512, 1, 1]

        x = x.view(x.size(0), -1) # [Batchsize, 512]
        
        x = self.classifier(x) # [Batchsize, num_classes]
        
        return x

In [3]:
vgg = VGG(10)

In [4]:
# state dictionary
a = {
    "key": 5,
    "name": "chunghyun",
    "age": 27
}

In [5]:
print(a["name"])

chunghyun


In [6]:
state_dict = vgg.state_dict()

In [7]:
type(state_dict)

collections.OrderedDict

In [8]:
for k, v in state_dict.items():
    print(k)

conv1_1.0.weight
conv1_1.0.bias
conv2_1.0.weight
conv2_1.0.bias
conv3_1.0.weight
conv3_1.0.bias
conv3_2.0.weight
conv3_2.0.bias
conv4_1.0.weight
conv4_1.0.bias
conv4_2.0.weight
conv4_2.0.bias
conv5_1.0.weight
conv5_1.0.bias
conv5_2.0.weight
conv5_2.0.bias
classifier.weight
classifier.bias


In [10]:
weight = state_dict["conv1_1.0.weight"]
print(weight.shape)
print(weight)

torch.Size([64, 3, 3, 3])
tensor([[[[-0.0764, -0.1197,  0.0123],
          [-0.0812,  0.0168, -0.1048],
          [ 0.0682,  0.1021, -0.0753]],

         [[ 0.0539, -0.1040,  0.1738],
          [ 0.1653,  0.1796, -0.0810],
          [-0.1722, -0.0299, -0.0277]],

         [[-0.1456, -0.1477, -0.1041],
          [-0.0983,  0.0875,  0.1176],
          [ 0.1666,  0.1655,  0.0710]]],


        [[[-0.1034, -0.0692,  0.0085],
          [ 0.1770,  0.0549,  0.1138],
          [-0.1111, -0.1614, -0.0655]],

         [[-0.1825, -0.0319,  0.1222],
          [-0.1783,  0.0717, -0.0194],
          [ 0.1332, -0.0024,  0.0123]],

         [[ 0.0257,  0.0768, -0.0205],
          [ 0.1155, -0.1794, -0.1061],
          [ 0.1037, -0.0285, -0.0885]]],


        [[[ 0.0725, -0.1410, -0.1742],
          [-0.1297,  0.1152, -0.1014],
          [ 0.1585,  0.0095,  0.1285]],

         [[-0.0097, -0.1815, -0.1713],
          [-0.0041, -0.1843,  0.1141],
          [-0.0743,  0.1402, -0.1052]],

         [[-0.1705

In [11]:
vgg = VGG(10).to(device)
vgg.train()
# print(vgg.state_dict())

VGG(
  (conv1_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv2_1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv3_1): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv3_2): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4_1): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv4_2): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv5_1): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (conv5_2): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (maxpool): MaxPool2d(

In [12]:
# Loading and normalizing CIFAR-10
import torch
import torchvision
import torchvision.transforms as transforms

import torch.optim as optim


transform_train = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalization value for cifar10
    ])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.parameters(), lr=0.001)

print(len(trainloader))

Files already downloaded and verified
Files already downloaded and verified
782


In [14]:
torch.save(vgg.state_dict(), "state_dict.pt")

In [15]:
state_dict_before = vgg.state_dict()

In [16]:
state_dict_after = torch.load("state_dict.pt")

In [19]:
print(state_dict_before['conv1_1.0.weight'])

tensor([[[[-0.1550,  0.1311, -0.0730],
          [-0.0310, -0.1659, -0.1092],
          [-0.1587, -0.0859,  0.0658]],

         [[-0.1543, -0.0692,  0.0299],
          [ 0.0873,  0.1903, -0.1028],
          [ 0.1657, -0.0163,  0.1028]],

         [[-0.1534, -0.0733,  0.1793],
          [ 0.1310, -0.0265, -0.1138],
          [ 0.0515,  0.1543,  0.0630]]],


        [[[-0.1426, -0.0045,  0.1683],
          [-0.1240, -0.0053, -0.0346],
          [-0.1391, -0.0825,  0.0780]],

         [[-0.0139, -0.0374, -0.1791],
          [-0.1222,  0.0160, -0.1572],
          [ 0.0093, -0.0191, -0.0382]],

         [[-0.1140,  0.0798,  0.1162],
          [ 0.0232,  0.0194,  0.0879],
          [-0.0213,  0.0805, -0.1212]]],


        [[[ 0.0552,  0.0595,  0.1751],
          [-0.1297, -0.0342,  0.0403],
          [ 0.0541, -0.0704,  0.1160]],

         [[ 0.1712,  0.1649, -0.1189],
          [-0.1422,  0.0098,  0.1151],
          [-0.1216, -0.0686,  0.0357]],

         [[ 0.0786, -0.0055, -0.0626],
     

In [20]:
print(state_dict_after['conv1_1.0.weight'])

tensor([[[[-0.1550,  0.1311, -0.0730],
          [-0.0310, -0.1659, -0.1092],
          [-0.1587, -0.0859,  0.0658]],

         [[-0.1543, -0.0692,  0.0299],
          [ 0.0873,  0.1903, -0.1028],
          [ 0.1657, -0.0163,  0.1028]],

         [[-0.1534, -0.0733,  0.1793],
          [ 0.1310, -0.0265, -0.1138],
          [ 0.0515,  0.1543,  0.0630]]],


        [[[-0.1426, -0.0045,  0.1683],
          [-0.1240, -0.0053, -0.0346],
          [-0.1391, -0.0825,  0.0780]],

         [[-0.0139, -0.0374, -0.1791],
          [-0.1222,  0.0160, -0.1572],
          [ 0.0093, -0.0191, -0.0382]],

         [[-0.1140,  0.0798,  0.1162],
          [ 0.0232,  0.0194,  0.0879],
          [-0.0213,  0.0805, -0.1212]]],


        [[[ 0.0552,  0.0595,  0.1751],
          [-0.1297, -0.0342,  0.0403],
          [ 0.0541, -0.0704,  0.1160]],

         [[ 0.1712,  0.1649, -0.1189],
          [-0.1422,  0.0098,  0.1151],
          [-0.1216, -0.0686,  0.0357]],

         [[ 0.0786, -0.0055, -0.0626],
     

In [34]:
### Train the network
for epoch in range(10):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # print statistics
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    
    #save models
    ##implement this
    os.makedirs('./checkpoint', exist_ok=True)
    torch.save(
        {
            'state_dict': vgg.state_dict(),
            'epoch': epoch
        },
        f'./checkpoint/vgg_epoch{epoch}.pt')
    torch.save(vgg.state_dict(), f'./checkpoint/vgg_latest.pt')
    
print('Finished Training')

[1,   100] loss: 0.675
[1,   200] loss: 0.664
[1,   300] loss: 0.662
[1,   400] loss: 0.676
[1,   500] loss: 0.688
[1,   600] loss: 0.695
[1,   700] loss: 0.665
[2,   100] loss: 0.544
[2,   200] loss: 0.581
[2,   300] loss: 0.571
[2,   400] loss: 0.601
[2,   500] loss: 0.573
[2,   600] loss: 0.598
[2,   700] loss: 0.605
[3,   100] loss: 0.467
[3,   200] loss: 0.469
[3,   300] loss: 0.495
[3,   400] loss: 0.517
[3,   500] loss: 0.490


KeyboardInterrupt: 

In [35]:
#load model
# checkpoint: ./checkpoint/vgg-epoch2.pt

vgg_after = VGG(10)
checkpoint = torch.load("./checkpoint/vgg_epoch0.pt")

In [39]:
vgg_after.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [40]:
vgg_after.state_dict()['conv1_1.0.weight'][0][0]

tensor([[-0.1228,  0.1161, -0.0619],
        [-0.0297, -0.1798, -0.0854],
        [-0.1226, -0.0380,  0.0923]])

In [41]:
checkpoint['state_dict']['conv1_1.0.weight'][0][0]

tensor([[-0.1228,  0.1161, -0.0619],
        [-0.0297, -0.1798, -0.0854],
        [-0.1226, -0.0380,  0.0923]], device='cuda:1')